In [4]:
#!/usr/bin/env python
"""
Monthly trading P&L table for a trading system.

Produces a Year x Month table where each cell is the net P&L for that month.

Two common definitions are supported:
  (A) Mark-to-market P&L (default): month-end portfolio value change (includes unrealized P&L)
  (B) Realized P&L (FIFO): sum of realized P&L from SELL executions in that month

Outputs (in OUTPUT_DIR):
  - 27_mtm_pnl_long.csv
  - 27_mtm_pnl_pivot.csv
  - 27_mtm_pnl_pivot.xlsx (best effort)
  - 27_realized_pnl_long.csv (if TRADES_FILE exists)
  - 27_realized_pnl_pivot.csv (if TRADES_FILE exists)
  - 27_realized_pnl_pivot.xlsx (best effort)
"""

import os
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

# ============================================================
# CONFIG
# ============================================================

TRADES_FILE = "./13-trading_output_regression_insp500_spyfilter_cap15/13-trades_regression_insp500_spyfilter_cap15.parquet"
EQUITY_FILE = "./13-trading_output_regression_insp500_spyfilter_cap15/13-equity_curve_regression_insp500_spyfilter_cap15.parquet"
OUTPUT_DIR  = "./25b-monthly_pnl_output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Equity curve value column candidates (update if your schema differs)
EQUITY_VALUE_COL_CANDIDATES = [
    "portfolio_value",
    "equity",
    "portfolio_after",
    "equity_after",
    "nav",
]

MONTH_LABELS = {1:"Jan", 2:"Feb", 3:"Mar", 4:"Apr", 5:"May", 6:"Jun",
                7:"Jul", 8:"Aug", 9:"Sep", 10:"Oct", 11:"Nov", 12:"Dec"}

print("=== MONTHLY P&L TABLE ===")

# ============================================================
# HELPERS
# ============================================================

def find_first_existing_col(df: pd.DataFrame, candidates: list[str]) -> str | None:
    for c in candidates:
        if c in df.columns:
            return c
    return None

def year_month_pivot(df_long: pd.DataFrame, value_col: str) -> pd.DataFrame:
    """Expect df_long has columns: year, month_name, value_col"""
    pivot = (
        df_long.pivot_table(
            index="year",
            columns="month_name",
            values=value_col,
            aggfunc="sum"  # for P&L we sum within month (long should already be one row/month)
        )
        .reindex(columns=list(MONTH_LABELS.values()))
        .sort_index()
    )
    return pivot

# ============================================================
# (A) MARK-TO-MARKET MONTHLY P&L FROM EQUITY CURVE
# ============================================================

if not os.path.exists(EQUITY_FILE):
    raise FileNotFoundError(f"EQUITY_FILE not found: {EQUITY_FILE}")

eq = pd.read_parquet(EQUITY_FILE).copy()

# normalize date
if "date" in eq.columns:
    eq["date"] = pd.to_datetime(eq["date"])
elif "exec_date" in eq.columns:
    eq["date"] = pd.to_datetime(eq["exec_date"])
else:
    raise ValueError("Equity file must contain a 'date' or 'exec_date' column.")

eq = eq.dropna(subset=["date"]).sort_values("date").reset_index(drop=True)

value_col = find_first_existing_col(eq, EQUITY_VALUE_COL_CANDIDATES)
if value_col is None:
    raise ValueError(
        f"Could not find a portfolio value column in equity curve. "
        f"Tried: {EQUITY_VALUE_COL_CANDIDATES}. "
        f"Available columns: {list(eq.columns)}"
    )

eq[value_col] = pd.to_numeric(eq[value_col], errors="coerce")
eq = eq.dropna(subset=[value_col]).copy()

# Dedup to last row per day (if multiple)
eq["day"] = eq["date"].dt.normalize()
eq = eq.groupby("day", as_index=False).tail(1).drop(columns=["day"]).sort_values("date").reset_index(drop=True)

start_date = eq["date"].min().normalize()
end_date   = eq["date"].max().normalize()
month_ends = pd.date_range(start=start_date, end=end_date, freq="M")
month_ends_df = pd.DataFrame({"month_end": month_ends})

# month-end values via merge_asof (last value on/before month_end)
eq_asof = eq.rename(columns={"date": "asof_date", value_col: "portfolio_value"}).sort_values("asof_date")
mtm = pd.merge_asof(
    month_ends_df.sort_values("month_end"),
    eq_asof[["asof_date", "portfolio_value"]],
    left_on="month_end",
    right_on="asof_date",
    direction="backward",
)

# drop leading months before first observation
mtm = mtm.dropna(subset=["portfolio_value"]).copy()

# Compute monthly P&L:
# - First month P&L = month_end_value - first available portfolio_value (start NAV)
start_nav = eq_asof["portfolio_value"].iloc[0]
mtm["prev_month_end_value"] = mtm["portfolio_value"].shift(1)
mtm["monthly_pnl"] = mtm["portfolio_value"] - mtm["prev_month_end_value"]
mtm.loc[mtm["prev_month_end_value"].isna(), "monthly_pnl"] = mtm.loc[mtm["prev_month_end_value"].isna(), "portfolio_value"] - start_nav

mtm["year"] = mtm["month_end"].dt.year
mtm["month"] = mtm["month_end"].dt.month
mtm["month_name"] = mtm["month"].map(MONTH_LABELS)

mtm_long = mtm[["month_end", "year", "month", "month_name", "portfolio_value", "monthly_pnl"]].copy()
mtm_pivot = year_month_pivot(mtm_long.rename(columns={"monthly_pnl": "value"}), value_col="value")

print("\n=== (A) MARK-TO-MARKET MONTHLY P&L (Year x Month) ===")
print(mtm_pivot.to_string(float_format=lambda x: f"{x:,.2f}"))

mtm_long.to_csv(os.path.join(OUTPUT_DIR, "27_mtm_pnl_long.csv"), index=False)
mtm_pivot.to_csv(os.path.join(OUTPUT_DIR, "27_mtm_pnl_pivot.csv"))
try:
    mtm_pivot.to_excel(os.path.join(OUTPUT_DIR, "27_mtm_pnl_pivot.xlsx"))
except Exception:
    pass

# ============================================================
# (B) REALIZED MONTHLY P&L FROM TRADES (FIFO PER SELL)
# ============================================================

def compute_realized_pnl_fifo(trades_df: pd.DataFrame) -> pd.DataFrame:
    """
    Realized P&L per SELL using FIFO matching against prior BUYs.
    Returns one record per SELL with realized_pnl and end_date.
    """
    records = []
    trades_df = trades_df.sort_values(["ticker", "date"]).reset_index(drop=True)

    for ticker, tdf in trades_df.groupby("ticker", sort=False):
        open_lots = []  # {shares, cost, date}

        for _, row in tdf.iterrows():
            side = str(row["type"]).upper()
            shares = float(row["shares"])
            price = float(row["price"])
            date = row["date"]

            if shares <= 0 or price <= 0:
                continue

            if side == "BUY":
                open_lots.append({"shares": shares, "cost": shares * price, "date": date})

            elif side == "SELL":
                if not open_lots:
                    continue

                remaining = shares
                total_cost = 0.0
                matched_shares = 0.0

                while remaining > 1e-8 and open_lots:
                    lot = open_lots[0]
                    take = min(remaining, lot["shares"])

                    cost_part = lot["cost"] * (take / lot["shares"])
                    total_cost += cost_part
                    matched_shares += take

                    lot["shares"] -= take
                    lot["cost"] -= cost_part
                    remaining -= take

                    if lot["shares"] <= 1e-8:
                        open_lots.pop(0)

                if matched_shares <= 1e-8:
                    continue

                fill_ratio = matched_shares / shares
                exit_value = (shares * price) * fill_ratio
                realized_pnl = exit_value - total_cost

                records.append({
                    "ticker": ticker,
                    "end_date": date,
                    "realized_pnl": realized_pnl,
                })

    return pd.DataFrame(records)

if os.path.exists(TRADES_FILE):
    trades = pd.read_parquet(TRADES_FILE).copy()

    # normalize date
    if "exec_date" in trades.columns:
        trades["date"] = pd.to_datetime(trades["exec_date"])
    elif "date" in trades.columns:
        trades["date"] = pd.to_datetime(trades["date"])
    else:
        raise ValueError("Trades file must contain 'exec_date' or 'date'.")

    required_cols = {"date", "ticker", "type", "shares", "price"}
    missing = required_cols - set(trades.columns)
    if missing:
        raise ValueError(f"Trades file missing required columns: {missing}")

    trades = trades.sort_values(["ticker", "date"]).reset_index(drop=True)

    realized = compute_realized_pnl_fifo(trades)
    if not realized.empty:
        realized["end_date"] = pd.to_datetime(realized["end_date"])
        realized["year"] = realized["end_date"].dt.year
        realized["month"] = realized["end_date"].dt.month
        realized["month_name"] = realized["month"].map(MONTH_LABELS)

        realized_monthly = (
            realized.groupby(["year", "month", "month_name"], as_index=False)["realized_pnl"]
            .sum()
            .rename(columns={"realized_pnl": "monthly_realized_pnl"})
        )

        realized_pivot = year_month_pivot(realized_monthly.rename(columns={"monthly_realized_pnl": "value"}),
                                          value_col="value")

        print("\n=== (B) REALIZED MONTHLY P&L (FIFO, Year x Month) ===")
        print(realized_pivot.to_string(float_format=lambda x: f"{x:,.2f}"))

        realized_monthly.to_csv(os.path.join(OUTPUT_DIR, "25b_realized_pnl_long.csv"), index=False)
        realized_pivot.to_csv(os.path.join(OUTPUT_DIR, "25b_realized_pnl_pivot.csv"))
        try:
            realized_pivot.to_excel(os.path.join(OUTPUT_DIR, "25b_realized_pnl_pivot.xlsx"))
        except Exception:
            pass
    else:
        print("\nNo realized P&L records computed from trades (no matched SELLs).")
else:
    print("\nTRADES_FILE not found — skipping realized monthly P&L.")

print(f"\nSaved outputs to: {OUTPUT_DIR}")
print("=== DONE ===")


=== MONTHLY P&L TABLE ===

=== (A) MARK-TO-MARKET MONTHLY P&L (Year x Month) ===
month_name         Jan          Feb         Mar         Apr          May         Jun         Jul         Aug         Sep         Oct         Nov         Dec
year                                                                                                                                                        
1999          9,553.98      -191.14   -8,630.87    7,238.24     6,024.09   12,977.90    3,536.88    8,691.24    3,416.20   16,884.10   30,522.86   48,622.69
2000         -8,450.83   128,814.54   21,265.44  -10,256.23   -19,510.64  -19,752.49   -5,753.56   32,540.52   46,391.80    1,095.17     -879.84    8,633.56
2001         -6,730.28         0.00        0.00        0.00         0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00
2002              0.00         0.00   -7,128.66    5,108.46    -9,629.04      573.06        0.00        0.00        0.00        0.00  